In [1]:
from decouple import config
import pymongo
from pprint import pprint
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import pandas as pd


MONGODB_API_KEY = config("MONGODB_API_KEY")
ANNICT_API_KEY = config("ANNICT_API_KEY")
ANNICT_API_URL = "https://api.annict.com/graphql"

# Select your transport with a defined url endpoint
headers = {
    "Content-type": "application/json",
    "Authorization": f"Bearer {ANNICT_API_KEY}"
    }
transport = AIOHTTPTransport(url=ANNICT_API_URL, headers=headers)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True, execute_timeout=300)




In [2]:
# Provide a GraphQL query
query = gql(
    """
query {
  searchWorks(
    orderBy: {field: WATCHERS_COUNT, direction: DESC},
  ) {
    nodes {
      annictId
      title
      titleKana
      watchersCount
      reviewsCount
      reviews {
        nodes {
          id
          ratingOverallState
          ratingMusicState
          ratingStoryState
          ratingAnimationState
          ratingCharacterState
        }
      }
    }
  }
}
"""
)

# result = client.execute(query)
result = await client.execute_async(query)

In [21]:
client = pymongo.MongoClient(MONGODB_API_KEY)
db = client.reviews
collection_raw_data = db.raw_data

In [3]:
df = pd.json_normalize(result["searchWorks"]["nodes"], ["reviews", "nodes"], ["annictId", "title", "titleKana", "watchersCount", "reviewsCount"])

In [12]:
df.to_pickle("./data/annictRaw.pkl")

In [4]:
df.head()

,id,ratingOverallState,ratingMusicState,ratingStoryState,ratingAnimationState,ratingCharacterState,annictId,title,titleKana,watchersCount,reviewsCount
0,UmV2aWV3LTIxNDE=,GREAT,GREAT,GREAT,GREAT,GREAT,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10750,101
1,UmV2aWV3LTIyNTM=,GOOD,GREAT,GOOD,AVERAGE,AVERAGE,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10750,101
2,UmV2aWV3LTI0MDI=,GREAT,GOOD,GREAT,GOOD,GREAT,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10750,101
3,UmV2aWV3LTE1MzI2,GOOD,GREAT,GREAT,BAD,GREAT,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10750,101
4,UmV2aWV3LTQ2Nzc3,GREAT,None,None,None,None,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10750,101


In [9]:
df.describe()

,id,ratingOverallState,ratingMusicState,ratingStoryState,ratingAnimationState,ratingCharacterState,annictId,title,titleKana,watchersCount,reviewsCount
count,60816,47543,38799,39095,39111,39095,60816,60816,60816,60816,60816
unique,60816,4,4,4,4,4,5221,5221,4372,2023,124
top,UmV2aWV3LTIxNDE=,GREAT,GOOD,GOOD,GREAT,GREAT,7713,邪神ちゃんドロップキックX,,928,6
freq,1,18905,14956,14128,15675,18375,275,275,2043,294,1387


In [13]:
df2 = df.replace({"BAD": -1, "AVERAGE": 1, "GOOD": 2, "GREAT": 4})

In [14]:
df2 = df2[df2["reviewsCount"]> 0]

In [15]:
df2.describe()

,ratingOverallState,ratingMusicState,ratingStoryState,ratingAnimationState,ratingCharacterState,annictId,watchersCount,reviewsCount
count,47543.000000,38799.000000,39095.000000,39111.000000,39095.000000,60816.000000,60816.000000,60816.000000
mean,2.521360,2.443491,2.319836,2.546700,2.713953,5295.476026,2285.357192,40.655469
std,1.331604,1.252575,1.411777,1.292015,1.316262,2298.433153,2106.670266,37.700087
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,2.000000,1.000000
25%,2.000000,1.000000,1.000000,2.000000,2.000000,3737.750000,729.000000,13.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,5624.000000,1666.000000,32.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000,7001.000000,3189.000000,56.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,10113.000000,10750.000000,275.000000


In [25]:
df_Overall = df2[["annictId", "id", "ratingOverallState", "title", "titleKana", "watchersCount", "reviewsCount"]]
df_Overall = df_Overall.dropna()
df_Music = df2[["annictId", "id", "ratingMusicState", "title", "titleKana", "watchersCount", "reviewsCount"]]
df_Music = df_Music.dropna()
df_Story = df2[["annictId", "id", "ratingStoryState", "title", "titleKana", "watchersCount", "reviewsCount"]]
df_Story = df_Story.dropna()
df_Animation = df2[["annictId", "id", "ratingAnimationState", "title", "titleKana", "watchersCount", "reviewsCount"]]
df_Animation = df_Animation.dropna()
df_Character = df2[["annictId", "id", "ratingCharacterState", "title", "titleKana", "watchersCount", "reviewsCount"]]
df_Character = df_Character.dropna()

In [26]:
df_Overall.describe()

,annictId,ratingOverallState,watchersCount,reviewsCount
count,47543.000000,47543.000000,47543.000000,47543.000000
mean,5370.774877,2.521360,2312.623940,38.414488
std,2374.173342,1.331604,2074.919463,32.481666
min,1.000000,-1.000000,2.000000,1.000000
25%,4147.000000,2.000000,779.000000,13.000000
50%,5788.000000,2.000000,1733.000000,31.000000
75%,7154.000000,4.000000,3187.000000,55.000000
max,10113.000000,4.000000,10750.000000,275.000000


In [30]:
collection_Overall = db.ratingOverallState
collection_Music = db.ratingMusicState
collection_Story = db.ratingStoryState
collection_Animation = db.ratingAnimationState
collection_Character = db.ratingCharacterState

In [28]:
collection_Overall.insert_many(df_Overall.to_dict("records"))

In [31]:
collection_Music.insert_many(df_Music.to_dict("records"))
collection_Story.insert_many(df_Story.to_dict("records"))
collection_Animation.insert_many(df_Animation.to_dict("records"))
collection_Character.insert_many(df_Character.to_dict("records"))
print("successfully inserted🍩")

successfully inserted🍩
